# OpenCVで動画の背景と動体を分離してみる

## アルゴリズム

 - 背景の抽出方法

    フレームに重みづけをしながら重ねていくと、動かない部分が浮かび上がってくる。
    単純に足すのではなく、現在のフレームを足しこんだ分、背景のフレームから引き算する。
    OpenCVでは、cv2.absdiff()を使う。また、重みづけ計算の誤差を減らすため、浮動小数点（np.float32）を用いて計算する。
    　
1. ゼロ埋めした背景フレームを用意する
1. 新しい背景 = 背景 × （１ － 重み） ＋ 現在のフレーム × 重み
1. ステップ２を繰り返す 　
        　

-  動体の抽出方法
    差分の絶対値が動体フレームになる。
    OpenCVでは、sv2.absdiff()を使う。

    　　動体フレーム = ｜現在のフレーム － 背景｜


In [8]:
import cv2
import numpy as np

# 定数定義
ESC_KEY = 27     # Escキー
INTERVAL = 33     # インターバル
FRAME_RATE = 30  # fps

WINDOW_ORG = "org"
WINDOW_BACK = "back"
WINDOW_DIFF = "diff"

FILE_ORG = "vtest.avi"

# ウィンドウの準備
cv2.namedWindow(WINDOW_ORG)
cv2.namedWindow(WINDOW_BACK)
cv2.namedWindow(WINDOW_DIFF)

# 元ビデオファイル読み込み
mov_org = cv2.VideoCapture(FILE_ORG)

# 最初のフレーム読み込み
has_next, i_frame = mov_org.read()

# 背景フレーム
back_frame = np.zeros_like(i_frame, np.float32)

# 変換処理ループ
while has_next == True:
    # 入力画像を浮動小数点型に変換
    f_frame = i_frame.astype(np.float32)

    # 差分計算
    diff_frame = cv2.absdiff(f_frame, back_frame)

    # 背景の更新
    cv2.accumulateWeighted(f_frame, back_frame, 0.025)

    # フレーム表示
    cv2.imshow(WINDOW_ORG, i_frame)
    cv2.imshow(WINDOW_BACK, back_frame.astype(np.uint8))
    cv2.imshow(WINDOW_DIFF, diff_frame.astype(np.uint8))

    # Escキーで終了
    key = cv2.waitKey(INTERVAL)
    if key == ESC_KEY:
        break

    # 次のフレーム読み込み
    has_next, i_frame = mov_org.read()

# 終了処理
cv2.destroyAllWindows()
mov_org.release()

## 閾値処理された画像上の輪郭を見つける

In [23]:
import cv2
import numpy as np

# 定数定義
ESC_KEY = 27     # Escキー
INTERVAL = 33     # インターバル
FRAME_RATE = 30  # fps

WINDOW_ORG = "org"
FILE_ORG = "vtest.avi"

# ウィンドウの準備
cv2.namedWindow(WINDOW_ORG)

# 元ビデオファイル読み込み
mov_org = cv2.VideoCapture(FILE_ORG)

# 最初のフレーム読み込み
has_next, frame = mov_org.read()

avg = None

# 変換処理ループ
while has_next == True:

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # 画像をグレースケールする

    if avg is None:
        avg = gray.copy().astype("float")
        continue

    cv2.accumulateWeighted(gray, avg, 0.5)  # 現在のフレームと移動平均との間の差を計算する
    frameDelta = cv2.absdiff(gray, cv2.convertScaleAbs(avg))

    thresh = cv2.threshold(frameDelta, 3, 255, cv2.THRESH_BINARY)[
        1]  # デルタ画像を閾値処理する

    cnts, _ = cv2.findContours(
        thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  # 閾値処理された画像上の輪郭を見つける

    min_area = 60
    cnts = [cnt for cnt in cnts if cv2.contourArea(cnt) > min_area]

    frame = cv2.drawContours(frame, cnts, -1, (0, 255, 0), 3)

    for cnt in cnts:
        (x, y), radius = cv2.minEnclosingCircle(cnt)
        center = (int(x), int(y))
        radius = int(radius)
        frame = cv2.circle(frame, center, 5, (255, 255, 255), -1)

    # フレーム表示
    cv2.imshow(WINDOW_ORG, frame)

    # Escキーで終了
    key = cv2.waitKey(INTERVAL)
    if key == ESC_KEY:
        break

    # 次のフレーム読み込み
    has_next, frame = mov_org.read()

# 終了処理
cv2.destroyAllWindows()
mov_org.release()